In [ ]:
Konteks: Screening Faktor dalam Proses Produksi Minuman Energi
Dalam produksi minuman energi, kita ingin mengidentifikasi faktor yang paling berpengaruh terhadap kandungan kafein (mg/L).
Kita mempertimbangkan 7 faktor dengan 2 level masing-masing:
A: Suhu Proses (°C) → (20, 30)
B: pH Larutan → (3.5, 4.5)
C: Waktu Pencampuran (menit) → (5, 10)
D: Konsentrasi Bahan A (%) → (1, 2)
E: Konsentrasi Bahan B (%) → (0.5, 1)
F: Kecepatan Pengadukan (rpm) → (100, 200)
G: Tekanan Pemrosesan (bar) → (1, 2)

Karena kita hanya ingin menyaring faktor utama, kita menggunakan Plackett-Burman Design, yang efisien untuk eksperimen dengan banyak faktor.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from pyDOE2 import pbdesign  # Library untuk Plackett-Burman

In [ ]:
# 1. Buat desain Plackett-Burman untuk 7 faktor
pb_matrix = pbdesign(7)
df = pd.DataFrame(pb_matrix, columns=["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
# 2. Konversi level (-1,1) ke nilai aktual
level_mapping = {
    "A": [20, 30],
    "B": [3.5, 4.5],
    "C": [5, 10],
    "D": [1, 2],
    "E": [0.5, 1],
    "F": [100, 200],
    "G": [1, 2]
}

In [ ]:
for col in df.columns:
    df[col] = df[col].map({-1: level_mapping[col][0], 1: level_mapping[col][1]})

In [ ]:
# 3. Simulasikan data response (Kandungan Kafein mg/L)
np.random.seed(42)
df["Kafein"] = (
    80 + 1.5 * (df["A"] - 20) - 3 * (df["B"] - 3.5) + 2 * (df["C"] - 5)
    + 0.5 * (df["D"] - 1) - 1 * (df["E"] - 0.5) + 0.8 * (df["F"] - 100)
    - 0.6 * (df["G"] - 1) + np.random.normal(0, 2, len(df))
)

In [ ]:
# 4. Analisis dengan model linier (ANOVA)
model = smf.ols('Kafein ~ A + B + C + D + E + F + G', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

In [ ]:
# 5. Visualisasi pengaruh faktor
plt.figure(figsize=(8,6))
sns.barplot(x=df.drop(columns=["Kafein"]).columns, y=model.params[1:].values)
plt.title("Pengaruh Faktor terhadap Kandungan Kafein")
plt.xlabel("Faktor")
plt.ylabel("Koefisien Efek")
plt.grid(True)
plt.show()

In [ ]:
# 6. Output hasil
print(df)
print("\nANOVA Table:\n", anova_table)
print("\nModel Summary:\n", model.summary())